In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM , Dropout , Dense

Using TensorFlow backend.


In [2]:
df_female = pd.read_csv('dataset/Female-Names.csv')
df_male = pd.read_csv('dataset/Male-Names.csv')

In [3]:
df_male.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [4]:
df = pd.concat([df_female , df_male])

In [5]:
df

,name,gender,race
0,shivani,f,indian
1,isha,f,indian
2,smt shyani devi,f,indian
3,divya,f,indian
4,mansi,f,indian
5,mazida,f,indian
6,pooja,f,indian
7,kajal,f,indian
8,meena,f,indian
9,sonam,f,indian


In [6]:
char2idx = {}
char_set = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
index = 0
for ch in char_set:
    char2idx[ch] = index
    index += 1
   

In [7]:
def should_keep(word):
    if len(word) > 19:
        return False
    for ch in word:
        if ch not in set(char_set):
            return False
    return True

In [8]:
def clean(word):
    word = str(word)
    char_set = [' ', '.','END', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
    name = word.lower()
    if should_keep(name):
        return name
    else:
        return None

In [9]:
df.name = df.name.apply(lambda x : clean(x))

In [10]:
df = df.dropna()

In [11]:
max_word_len = 20
def convert_one_hot(word):
    word_chars = []
    
    
    for ch in word:
        one_hot_vec = np.zeros(len(char_set),)
        one_hot_vec[char2idx[ch]] = 1
        word_chars.append(one_hot_vec)
    
    
    for _ in range(max_word_len-len(word)):
        one_hot_vec = np.zeros(len(char_set),)
        one_hot_vec[char2idx['END']] = 1
        word_chars.append(one_hot_vec)
    
    
    word_chars = np.array(word_chars)
    return word_chars

In [12]:
words = []
labels = []
for n , g in df.drop('race' , axis = 1).itertuples(index=False):
    words.append(convert_one_hot(n))
    labels.append(g)

In [13]:
words = np.array(words)
labels = np.array(labels)

In [14]:
words.shape

(27980, 20, 29)

In [15]:
ohe = OneHotEncoder()
y = ohe.fit_transform(labels.reshape(-1 , 1)).todense()

In [16]:
y[:10]

matrix([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

In [30]:
# model = Sequential()
# model.add(LSTM(64 , input_shape = (20 , 29)  , return_sequences = True))
# model.add(LSTM(64))
# model.add(Dropout(.3))
# model.add(Dense(2 , activation='sigmoid'))
# model.summary()


In [32]:
# model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [33]:
model = Sequential()
model.add(LSTM(128 , input_shape = (20 , 29)  , return_sequences = True))
model.add(LSTM(64))
model.add(Dropout(.3))
model.add(Dense(2 , activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 20, 128)           80896     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 130,434
Trainable params: 130,434
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer='rmsprop' , loss = 'binary_crossentropy' , metrics = ['acc'])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(words, y, test_size=0.33, random_state=42)

In [26]:
model.fit(X_train , y_train , epochs = 10)

Epoch 1/10
18746/18746 [==============================] - 23s 1ms/step - loss: 0.5222 - acc: 0.7433: 0s - loss: 0.5241 - acc: 
Epoch 2/10
18746/18746 [==============================] - 21s 1ms/step - loss: 0.3847 - acc: 0.8364
Epoch 3/10
18746/18746 [==============================] - 22s 1ms/step - loss: 0.3210 - acc: 0.8734
Epoch 4/10
18746/18746 [==============================] - 21s 1ms/step - loss: 0.2805 - acc: 0.8921
Epoch 5/10
18746/18746 [==============================] - 21s 1ms/step - loss: 0.2556 - acc: 0.9057
Epoch 6/10
18746/18746 [==============================] - 21s 1ms/step - loss: 0.2371 - acc: 0.9166
Epoch 7/10
18746/18746 [==============================] - 21s 1ms/step - loss: 0.2232 - acc: 0.9213
Epoch 8/10
18746/18746 [==============================] - 20s 1ms/step - loss: 0.2078 - acc: 0.9286
Epoch 9/10
18746/18746 [==============================] - 20s 1ms/step - loss: 0.1962 - acc: 0.9333
Epoch 10/10
18746/18746 [==============================] - 21s 1ms/step -

In [27]:
model.evaluate(X_test , y_test)

9234/9234 [==============================] - 4s 389us/step


[0.2182982064071919, 0.9285250162572243]

In [28]:
def predict(name):
  gen=np.argmax(model.predict(convert_one_hot(name).reshape(1,20,29)))
  if gen==0:
    return 'f'
  else:
    return 'm'

In [29]:
names = ['shreya','anuj','lovely','sweety']
for name in names:
  gender=predict(name)
  print("Name :", name," Gender :", gender)

Name : shreya  Gender : f
Name : anuj  Gender : m
Name : lovely  Gender : f
Name : sweety  Gender : f
